# 2. Setting up the Environment

In [1]:
from IPython.display import clear_output

'''
!git clone https://github.com/matterport/Mask_RCNN.git # load Mask R-CNN code implementation
!git clone https://github.com/leekunhee/Mask_RCNN # load Mask R-CNN code implementation (tensorflow2.2 compatible)
!git clone https://github.com/ruslan-kl/brain-tumor.git # load new data set and annotations
!git clone https://github.com/Anushaaelango/brain-tumor # load new data set and annotations (available)
!pip3 install pycocotools

rm -rf brain-tumor/.git/
rm -rf Mask_RCNN/.git/
'''

clear_output()

In [2]:
import os 
import sys
from tqdm import tqdm
import cv2
import numpy as np
import json
import skimage.draw
import matplotlib
import matplotlib.pyplot as plt
import random

# Root directory of the project
ROOT_DIR = os.path.abspath('Mask_RCNN/')

# Import Mask RCNN
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.model import log

import mrcnn.model as modellib
from mrcnn import visualize

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, 'samples/coco/'))

from Mask_RCNN.samples.coco import coco

plt.rcParams['figure.facecolor'] = 'white'

clear_output()

In [3]:
def get_ax(rows=1, cols=1, size=7):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [4]:

MODEL_DIR = os.path.join(ROOT_DIR, 'logs') # directory to save logs and trained model
# ANNOTATIONS_DIR = 'brain-tumor/data/new/annotations/' # directory with annotations for train/val sets
DATASET_DIR = 'brain-tumor/data_cleaned/' # directory with image data
DEFAULT_LOGS_DIR = 'logs' 

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed

if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)    


In [5]:
from tumor_config import TumorConfig
    
config = TumorConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.85
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INST

In [6]:
from brain_scan_dataset import BrainScanDataset

# 3. Mask R-CNN Model

In [7]:
model = modellib.MaskRCNN(
    mode='training', 
    config=config, 
    model_dir=DEFAULT_LOGS_DIR
)

model.load_weights(
    COCO_MODEL_PATH, 
    by_name=True, 
    exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"]
)

2023-11-04 20:19:07.619437: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-04 20:19:07.643385: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'proposal_targets/cond/output/_16'


In [8]:
# Training dataset.
dataset_train = BrainScanDataset()
dataset_train.load_brain_scan(DATASET_DIR, 'train')
dataset_train.prepare()

# Validation dataset
dataset_val = BrainScanDataset()
dataset_val.load_brain_scan(DATASET_DIR, 'val')
dataset_val.prepare()

dataset_test = BrainScanDataset()
dataset_test.load_brain_scan(DATASET_DIR, 'test')
dataset_test.prepare()

# Since we're using a very small dataset, and starting from
# COCO trained weights, we don't need to train too long. Also,
# no need to train all layers, just the heads should do it.
print("Training network heads")
model.train(
    dataset_train, dataset_val,
    learning_rate=config.LEARNING_RATE,
    epochs=15,
    layers='heads'
)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: logs/tumor_detector20231104T2019/mask_rcnn_tumor_detector_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcn

/home/sdp/.local/lib/python3.10/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-11-04 20:19:09.069052: W tensorflow/c/c_api.cc:305] Operation '{name:'rpn_bbox_pred/bias/Assign' id:5724 op device:{requested: '', assigned: ''} def:{{{node rpn_bbox_pred/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](rpn_bbox_pred/bias, rpn_bbox_pred/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-04 20:19:10.875202: W tensorflow/c/c_api.cc:305] Operation '{name:'range' id:9905 op device:{requested: '', assigned: ''} def:{{{node range}} = Range[Tidx=DT_INT32, _has_manual_control_dependencies=true](range/start, Rank, ran

Epoch 1/15


2023-11-04 20:19:29.483553: W tensorflow/c/c_api.cc:305] Operation '{name:'loss/AddN' id:11024 op device:{requested: '', assigned: ''} def:{{{node loss/AddN}} = AddN[N=5, T=DT_FLOAT, _has_manual_control_dependencies=true](mul, mul_1, mul_2, mul_3, mul_4)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-04 20:19:29.720432: W tensorflow/c/c_api.cc:305] Operation '{name:'training/SGD/mrcnn_class_bn1/gamma/momentum/Assign' id:12977 op device:{requested: '', assigned: ''} def:{{{node training/SGD/mrcnn_class_bn1/gamma/momentum/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/SGD/mrcnn_class_bn1/gamma/momentum, training/SGD/mrcnn_class_bn1/gamma/momentum/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will h

  1/100 [..............................] - ETA: 9:21 - batch: 0.0000e+00 - size: 1.0000 - loss: 22.4519 - rpn_class_loss: 14.6753 - rpn_bbox_loss: 7.7765 - mrcnn_class_loss: 7.7721e-05 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00

...

  2/100 [..............................] - ETA: 1:11 - batch: 0.5000 - size: 1.0000 - loss: 18.0890 - rpn_class_loss: 11.5956 - rpn_bbox_loss: 6.4933 - mrcnn_class_loss: 6.1033e-05 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00    

2023-11-04 20:19:33.030659: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2023-11-04 20:19:33.137120: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2023-11-04 20:19:33.138309: I tensorflow/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/tumor_detector20231104T2019/plugins/profile/2023_11_04_20_19_33/training.xplane.pb


100/100 [==============================] - ETA: 0s - batch: 49.5000 - size: 1.0000 - loss: 5.6169 - rpn_class_loss: 0.2791 - rpn_bbox_loss: 3.7054 - mrcnn_class_loss: 0.0312 - mrcnn_bbox_loss: 1.2937 - mrcnn_mask_loss: 0.3074

/home/sdp/.local/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2023-11-04 20:21:40.063993: W tensorflow/c/c_api.cc:305] Operation '{name:'loss/AddN' id:11024 op device:{requested: '', assigned: ''} def:{{{node loss/AddN}} = AddN[N=5, T=DT_FLOAT, _has_manual_control_dependencies=true](mul, mul_1, mul_2, mul_3, mul_4)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-04 20:21:40.763047: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key:

100/100 [==============================] - 244s 2s/step - batch: 49.5000 - size: 1.0000 - loss: 5.6169 - rpn_class_loss: 0.2791 - rpn_bbox_loss: 3.7054 - mrcnn_class_loss: 0.0312 - mrcnn_bbox_loss: 1.2937 - mrcnn_mask_loss: 0.3074 - val_loss: 1.1941 - val_rpn_class_loss: 0.0748 - val_rpn_bbox_loss: 1.1193 - val_mrcnn_class_loss: 7.1525e-06 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 2/15
100/100 [==============================] - 274s 3s/step - batch: 49.5000 - size: 1.0000 - loss: 1.9256 - rpn_class_loss: 0.0060 - rpn_bbox_loss: 0.8648 - mrcnn_class_loss: 0.0354 - mrcnn_bbox_loss: 0.6211 - mrcnn_mask_loss: 0.3983 - val_loss: 0.5346 - val_rpn_class_loss: 0.0213 - val_rpn_bbox_loss: 0.5133 - val_mrcnn_class_loss: 1.3351e-05 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 3/15
100/100 [==============================] - 287s 3s/step - batch: 49.5000 - size: 1.0000 - loss: 0.6414 - rpn_class_loss: 0.0037 - rpn_bbox_loss: 0.2150 - mrcn